### 참고

- 넷플릭스
    - 슬로건 : 모든 것이 추천이다.
    - https://medium.com/netflixtechblog/55838468f429
    - 넷플릭스의 추천 알고리즘 구현에 대한 기술 블로그 공개

### 1. 연구 목표

- 사용자 평점 데이터를 기반으로 사용자 특정을 예측하여 추천 시스템을 구축한다.
- 실제 OTT, 쇼핑몰에서 회원가입 시 추천 알고리즘으로 사용된다.
- 회귀 처리, 회귀 평가, 추천시스템에 대한 이해
- FastFM(third party 알고리즘 사용 -> 인수분해머신 기능 지원)
    - 윈도우에서 컴파일 후 설치가 불가능하므로 리눅스에 설치하여 개발을 진행

### 2. 데이터 수집/확보

- ml-100k.zip 파일 제공
    - 미국에 있는 movelens라는 사이트에서 제공
- 영화 정보 데이터
    - 고객 정보 : u.user
    - 영화 정보 : u.item
    - 평점 정보 : u.data

In [8]:
import pandas as pd

In [13]:
# 유저 정보 : u.user
# u.user라는 데이터에 컬럼이 없다보니, 첫번째 데이터가 컬럼이 되었다.
# 이를 예방하기 위해서는 컬럼을 지정한다.
cols = ['uid', 'age', 'm', 'job', 'zip_code']
users = pd.read_csv('./table/ml-100k/u.user', sep='|', names=cols)

In [15]:
users.head(2)

,uid,age,m,job,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043


In [16]:
users.tail(2)

,uid,age,m,job,zip_code
941,942,48,F,librarian,78209
942,943,22,M,student,77841


In [26]:
# 고객 943명의 데이터
users.shape

(943, 5)

In [24]:
# 영화 정보 로드 : u.item
m_cols = ['mid', 'title', 'release_date', 'video_release_date', 'imdb_url']

# 영어권이라면 utf-8에서 에러나면 latin1으로 사용
# 원본 데이터의 컬럼이 많은데 부분만 쓰고 싶다면 usecols를 적용
movies = pd.read_csv('./table/ml-100k/u.item', sep='|', encoding='latin1', names=m_cols, usecols=range(5))
movies.head(2)

,mid,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...


In [27]:
movies.tail(2)

,mid,title,release_date,video_release_date,imdb_url
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...
1681,1682,Scream of Stone (Schrei aus Stein) (1991),08-Mar-1996,NaN,http://us.imdb.com/M/title-exact?Schrei%20aus%...


In [28]:
movies.shape

(1682, 5)

In [30]:
movies.isnull().sum()

mid                      0
title                    0
release_date             1
video_release_date    1682
imdb_url                 3
dtype: int64

In [33]:
# 평점 정보 : u.data
r_cols = ['uid', 'mid', 'rating', 'unix_timestamp']
ratings = pd.read_csv('./table/ml-100k/u.data', sep='\t', names=r_cols)
ratings.head(2)

,uid,mid,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [34]:
ratings.tail(2)

,uid,mid,rating,unix_timestamp
99998,13,225,2,882399156
99999,12,203,3,879959583


In [35]:
ratings.shape

(100000, 4)

In [39]:
# unix_timestamp는 1970년 1월 1일 00시 00분 00초 부터 현재까지 경과된 시간 + 09:00
# 그래서 현재시간 특정시간을 특정하기가 어렵다. -> 해석이 어렵다. -> 시간형식 변경
ratings['date'] = pd.to_datetime(ratings.unix_timestamp, unit='s')
ratings.head(2)

,uid,mid,rating,unix_timestamp,date
0,196,242,3,881250949,1997-12-04 15:55:49
1,186,302,3,891717742,1998-04-04 19:22:22


In [40]:
ratings.tail(2)

,uid,mid,rating,unix_timestamp,date
99998,13,225,2,882399156,1997-12-17 22:52:36
99999,12,203,3,879959583,1997-11-19 17:13:03


In [41]:
ratings.shape

(100000, 5)

In [ ]:
# 3. 데이터 준비/품질향상/전처리

In [ ]:
# 4. 데이터 분석/통계적, 시각적, ...

In [ ]:
# 5. 예측 모델 구축(머신러닝 기반) 

In [ ]:
# 6. 시스템 통합(서비스에 반영, OTT, 쇼핑몰 등 대상)